In [85]:
# import packages
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns  
import statsmodels.api as sm

In [86]:
# Import data
path = Path('/Users/thuymle/Dropbox/SIEPR Projects/Inflation and Election Swings')
input = path / 'input'
output = path / 'output'
data = path / 'data'
election_data = (data / 'election_data')
inflation_data = (data / 'inflation_data')

### Inflation Data

In [87]:
# MSA Level Data for Inflation
raw_bea_msa = pd.read_excel(inflation_data / 'raw/bea_msa_rpp.xlsx', skiprows=5)

# Data Prep for merging
msa_rpp_df = raw_bea_msa.copy() 
msa_rpp_df = msa_rpp_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

# generate 2020-2022 cumulative inflation (percent change in RPP):
msa_rpp_df['rpp_change_20_22'] = ((msa_rpp_df['2022'] - msa_rpp_df['2020']) / msa_rpp_df['2020'] ) * 100

# Get the right MSA format
msa_rpp_df['msa'] = msa_rpp_df['GeoName'].str.split(',').str[0]
msa_rpp_df['state'] = msa_rpp_df['GeoName'].str.split(',').str[1].str.strip().str.split().str[0]
msa_rpp_df['msa_short'] = msa_rpp_df['msa'].str.split('-').str[0]
msa_rpp_df['state_short'] = msa_rpp_df['state'].str.split('-').str[0]

#Get Categories
recode_categories_dict = {
    'rpps: all items': 'all items',
    '  rpps: goods': 'goods',
    '  rpps: services: housing': 'housing',
    '  rpps: services: utilities': 'utilities',
    '  rpps: services: other': 'other services'
}
msa_rpp_df['Description'] = msa_rpp_df['Description'].replace(recode_categories_dict)
msa_rpp_df.drop(columns=['LineCode'], inplace=True)

# re-name columns
msa_rpp_df.rename(columns = {
    'Description': 'category',
    'GeoName': 'msa_full'
    }, inplace=True)

msa_rpp_df['msa'] = msa_rpp_df['msa_short'] + ', ' + msa_rpp_df['state_short']
keep = ['msa', 'category', 'rpp_change_20_22', 'msa_full']
msa_inflation_bea = msa_rpp_df[keep]
# msa_rpp_df.to_csv(f'{inflation_data}/clean/bea_msa_inflation.csv', index=False)

/var/folders/vd/gxvvvrrd6lb4t053j12d3y100000gn/T/ipykernel_18735/1576621564.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  msa_rpp_df = msa_rpp_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


### Vote Swing and Housing Index

In [88]:
# Vote swing data prep
file = output / "vote_swing.csv"
df_election = pd.read_csv(file)
df_election.drop(columns = 'msa', inplace = True)
df_election.rename(columns = {'regionname': 'msa'}, inplace = True)


# Zillow 2024 index prep
zillow_rent = input / 'zillow_median_rent_raw.csv'
df_zillow = pd.read_csv(zillow_rent)
df_zillow.columns = df_zillow.columns.str.lower()
df_zillow['regionname'] = df_zillow['regionname'].str.lower()

keep = ['regionname', '10/31/2024']
df_zillow = df_zillow[keep]
df_zillow.rename(columns = {'regionname': 'msa'}, inplace = True)

# Merge zillow and elecuib
zillow_election = pd.merge(df_zillow, df_election, on = 'msa', how = 'outer')
zillow_election.rename(columns = {'10/31/2024': 'zillow_index_2024'}, inplace = True)
print(zillow_election.columns)

Index(['msa', 'zillow_index_2024', 'vote_swing'], dtype='object')


## Creating Master Dataset

In [89]:
# Merge
regression_data = pd.merge(zillow_election, msa_inflation_bea, on = 'msa', how = 'outer')
# Keep only all items rows
regression_data = regression_data.loc[regression_data['category'] == 'goods']
regression_data.drop(columns = {'msa_full', 'category'}, inplace = True)
# regression_data.drop(columns = {'msa_full'}, inplace = True)

# Drop missing values
regression_data.dropna(inplace = True)

In [90]:
# regression_data['interaction'] = regression_data['zillow_index_2024'] * regression_data['rpp_change_20_22']

# def run_regression(group):
#     X = group[['rpp_change_20_22', 'zillow_index_2024', 'interaction']]
#     X = sm.add_constant(X)
#     y = group['vote_swing']

#     model = sm.OLS(y, X).fit()

#     print(f"Category: {group['category'].iloc[0]}")
#     print(model.summary())
#     print("\n")

# regression_data.groupby('category').apply(run_regression)

In [91]:
# Cross-Sectional Regression with Inflation and House Price Index
regression_data['interaction'] = regression_data['zillow_index_2024'] * regression_data['rpp_change_20_22']

X = regression_data[['rpp_change_20_22', 'zillow_index_2024', 'interaction']]
X = sm.add_constant(X)
y = regression_data['vote_swing'] 
# Fit the OLS regression model
model = sm.OLS(y, X).fit()

# Display regression results
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:             vote_swing   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     3.381
Date:                Fri, 06 Dec 2024   Prob (F-statistic):             0.0185
Time:                        15:33:12   Log-Likelihood:                -710.07
No. Observations:                 348   AIC:                             1428.
Df Residuals:                     344   BIC:                             1444.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.9672      0.40

In [92]:
X = regression_data[['rpp_change_20_22', 'zillow_index_2024', 'interaction']]
X = sm.add_constant(X)
y = regression_data['vote_swing']

# Fit the OLS regression model
model = sm.OLS(y, X).fit()

# Extract results into a DataFrame
results_df = pd.DataFrame({
    'Coefficient': model.params,
    'Std. Error': model.bse,
    'P-value': model.pvalues,
    '95% CI Lower': model.conf_int()[0],
    '95% CI Upper': model.conf_int()[1]
})
results_df.index.name = 'Variable'

# Print the DataFrame to verify
print(results_df)

# Convert the DataFrame to LaTeX
latex_table = results_df.to_latex(
    float_format="{:.4f}".format,  # Format numerical values to 4 decimal places
    caption="OLS Regression Results",  # Add a caption
    label="tab:regression_results",    # Add a label for referencing in LaTeX
    column_format="lcccc",            # Align columns (e.g., left, center)
    escape=False                      # Avoid escaping characters (e.g., if special symbols exist)
)

# Save the LaTeX table to a file
with open("regression_results.tex", "w") as f:
    f.write(latex_table)

print("LaTeX table saved to regression_results.tex")



                   Coefficient  Std. Error   P-value  95% CI Lower  \
Variable                                                             
const                 0.967237    0.401566  0.016536      0.177404   
rpp_change_20_22     -0.059681    0.162890  0.714298     -0.380066   
zillow_index_2024     0.000709    0.000245  0.003992      0.000228   
interaction           0.000013    0.000090  0.888474     -0.000164   

                   95% CI Upper  
Variable                         
const                  1.757070  
rpp_change_20_22       0.260703  
zillow_index_2024      0.001191  
interaction            0.000189  
LaTeX table saved to regression_results.tex
